# ТЗ

На последнем семинаре мы проанализировали несколько различных морфологических теггеров. Как же узнать, какой использовать? Давайте сравним их качество!

В этой домашке вам будет нужно найти тексты на русском языке (размер корпуса не менее 200 слов), в которых будут какие-то трудные или неоднозначные для POS теггинга моменты и разметить их вручную – с помощью этих текстов мы будем оценивать качество работы наших теггеров. В текстах размечаем только части речи, ничего больше!

(1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.

(3 балла) Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.

(2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.

(4 балла) Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за создание такого рода чанкера, балл за за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без.

### FAQ
В каком формате должна быть ручная разметка корпуса? - В любом, как вам удобно (табличка токен + тег, conllu, и тд). Не забудьте загрузить его на гитхаб вместе с решением.

Верно ли, что корпусом может быть набор не связанных между собой предложений? - Да.

Если случай спорный, у него может быть два разбора или нужно принять единое решение? - Может быть два разбора, если так подсказывает ваша лингвистическая интуиция.

Как оценивать accuracy, если POS-теггер не разрешает омонимию? - Проверить, что это точно так. За правильные ответы тогда можно считать случаи, когда тег из gold-разметки есть в ответе теггера, либо как это часто бывает, брать первый (нулевой) вариант - you decide.

Что делать со штуками вроде причастия? Если теггер указывает его как глагол, а мы разметили как причастие, это стоит засчитать (потому что у теггера не было возможности узнать наш тегсет) или нет? Или как раз на моменте выбора тегсета нужно продумать его так, чтобы по возможности избежать таких ситуаций? - Можно при конвертации тегов посмотреть, не нужно ли учитывать для этого теггера не только POS-тег, но и какие-то другие граммемы, чтобы потом преобразовать этот тег в причастие. Собственно, для таких случаев и нужна какая-то более хитрая конвертация, а не просто словарик соответствий POS-тегов.



# Концепт

Прочитав про корпус нестандартных слов, я в ту же секунду подумала о всяких словотворческих экспериментах вроде хрюкотающих (*хрюкочущих?.. кажется, я тут не сильно лучше морфопарсера*) зелюков в мове и глокой куздры. 

Но я решила пожалеть себя и парсеры и взять что-то, обладающее некоторым количеством реальных слов. Посему мой корпус состоит исключительно из стихотворений Велимира Хлебникова (не сильно лучше и не очень оригинально, согласна).

Почему этот корпус более-менее ок для теста разметки?
- выдуманные и редкие слова (*в вырей стаи времирей умчались*, *осокоревые кущи*)
- слова с дефисами (в том числе и выдуманные слова с дефисами) 
- некоторое количество омонимов

# Импорты и первичная токенизация 

In [1]:
import nltk
from nltk import sent_tokenize, word_tokenize
import csv
from string import punctuation
import pymorphy2
import pandas as pd
from sklearn.metrics import accuracy_score
import spacy

In [2]:
from pymystem3 import Mystem

In [4]:
morph = pymorphy2.MorphAnalyzer()
mstem = Mystem()

In [21]:
# здесь просто был код для того, чтобы побить текст на токены

# poems_file= open('poems.txt', 'r', encoding='UTF-8')
# poems_text = poems_file.read()
# tokens = [word.lower() for word in word_tokenize(poems_text) if word not in punctuation]
# with open('markup.csv', 'w', newline='', encoding="UTF-8") as file:
#     writer = csv.writer(file)
#     for token in tokens:
#         writer.writerow([token])

 # Тэгсет
 Вот мой общий тэгсет, в который я планирую радостно переводить все полученные всеми тэггерами тэги.
 Частично он был вдохновлён предыдущим соревновательным ридингом, частично чем-то, что мне иногда кажется здравым смыслом.
 
 NOUN - существительные,
 
 ADJ - прилагательные,
 
 VERB - глаголы, а ещё причастия и деепричастия,
 
 ADV - наречия, а ещё частицы и междометия,
 
 PRON - местоимения,
 
 PREP - предлоги, 
 
 CONJ - союзы,
 
 NUM - числительные
 
 Я просто посмотрела на тэгсеты pymorphy, spacy и mystem и решила собрать из них один единый словарь значений, соответствующих моим выделенным тэгам, это показалось логичнее, чем прописывать три отдельных словаря/три отдельные функции.
 
<!--     
pymorphy: 
PARTS_OF_SPEECH = frozenset([
        'NOUN',  # имя существительное
        'ADJF',  # имя прилагательное (полное)
        'ADJS',  # имя прилагательное (краткое)
        'COMP',  # компаратив
        'VERB',  # глагол (личная форма)
        'INFN',  # глагол (инфинитив)
        'PRTF',  # причастие (полное)
        'PRTS',  # причастие (краткое)
        'GRND',  # деепричастие
        'NUMR',  # числительное
        'ADVB',  # наречие
        'NPRO',  # местоимение-существительное
        'PRED',  # предикатив
        'PREP',  # предлог
        'CONJ',  # союз
        'PRCL',  # частица
        'INTJ',  # междометие
    ])
    
    
spacy: 
POS_LIST = ["ADJ", "ADP", "ADV", "AUX", "CONJ", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X", "SPACE"]
    
mystem:  
A прилагательное
ADV наречие
ADVPRO местоименное наречие
ANUM числительное-прилагательное
APRO местоимение-прилагательное
COM часть композита - сложного слова
CONJ союз
INTJ междометие
NUM числительное
PART частица
PR предлог
S существительное
SPRO местоимение-существительное
V глагол -->

In [138]:
convert_dict = {'NOUN':['NOUN', 'PROPN', 'S'], 
                'ADJ': ['ADJF', 'ADJS', 'COMP', 'ADJ', 'A'],
                'VERB': ['VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'AUX', 'V'],
                'ADV': ['ADVB', 'PRCL', 'INTJ', 'ADV', 'PART'], 
                'PRON': ['NPRO', 'DET', 'PRON', 'ADVPRO', 'APRO', 'SPRO'], 
                'PREP': ['PREP', 'ADP', 'PR'],
                'CONJ': ['CONJ', 'CCONJ', 'SCONJ'],
                'NUM': ['NUMR', 'NUM', 'ANUM']}

In [139]:
def convert_tags(tag_list):
    converted_tags = []
    for tag in tag_list:
        for ult_tag, tag_options in convert_dict.items():
            if tag in tag_options:
                converted_tags.append(ult_tag)
                break
        else:
            converted_tags.append(tag)
    return converted_tags


print(convert_tag(['A', 'SPRO', 'SCONJ'])) # просто тестовый принт для проверки работы

['ADJ', 'PRON', 'CONJ']


In [140]:
tokens_list = []
annot_pos = []
with open('markup.csv', 'r', encoding = "UTF-8") as f: 
    file = csv.reader(f)
    for row in file: 
        tokens_list.append(row[0])
        annot_pos.append(row[1])

# Pymorphy

Дальше для каждого из выбранных тэггеров всё будет выглядеть примерно одинаково. 

Я паршу токены, вылавливаю оттуда POS-тэг, записываю в лист тэгов и конвертирую этот лист во что-то более универсальное с помощью ранее сделанного словарика тэгов.

In [141]:
pmrph_tags = []
for token in tokens_list:
    parsed = morph.parse(token)
    pmrph_tags.append(str(parsed[0].tag.POS))

In [142]:
pymorphy_tags = convert_tags(pmrph_tags)

# Spacy

In [143]:
!python -m spacy download ru_core_news_md

     --------------------------------------- 41.9/41.9 MB 31.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [144]:
spacy_tags = []
nlp = spacy.load("ru_core_news_md")
doc = nlp(' '.join(tokens_list))
for token in doc:
    spacy_tags.append(token.pos_)

In [145]:
spacy_tags = convert_tags(spacy_pos)

# Mystem

In [146]:
a = mstem.analyze(' '.join(tokens_list)) 

In [147]:
mystem_tags = []
words_annotated = []
for word in a:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        mystem_tags.append(pos)
        words_annotated.append([word['text'], pos])

In [148]:
mystem_tags = convert_tags(mystem_tags)

# Accuracy

In [149]:
print("Pymorphy:", accuracy_score(pymorphy_tags, annot_pos))
print("Spacy:", accuracy_score(spacy_tags, annot_pos))
print("Mystem:", accuracy_score(mystem_tags, annot_pos))

Pymorphy: 0.9122137404580153
Spacy: 0.8664122137404581
Mystem: 0.9541984732824428


# Chunking

Лучшим тэггером, в итоге, оказался Mystem. Это немножко грустно, поскольку у него очень неудобный формат выдачи, потому придётся пострадать (и попробовать отредактировать один из предыдущих кусков кода так, что у нас будет список, маппящий слова к тэгам). 

(Тут я уже исходила из того, что конвертация тэгов нам не очень нужна, потому довольствуемся майстемовским тэгсетом)

In [150]:
words_annotated[:5] # сниппет того, как выглядит пословная разметка

[['в', 'PR'], ['пору', 'S'], ['когда', 'CONJ'], ['в', 'PR'], ['вырей', 'S']]

Напишу простой и относительно красивый чанкер только для частей речи.

In [151]:
def chunker(tag1, tag2):
    bigram_list = []
    for word in words_annotated:
        if tag1 == word[1]:
            next_word = words_annotated[words_annotated.index(word)+1] # поиск по индексу следующего элемента
            if next_word[1] == tag2:
                bigram = word[0] + ' ' + next_word[0]
                bigram_list.append(bigram)
    return bigram_list

In [152]:
print(chunker('A', 'S'))

['томный круг', 'нежная дитяти', 'небесных прав', 'туманных веко', 'малиновому рту', 'осокоревые кущи', 'суровым садом', 'черноокой горожанки', 'медлительной южанки', 'белого казани', 'низок севастополь', 'целый год', 'гордый тополь', 'прямодушнее туркмена', 'милая измена', 'звонким смехом']


Теперь напишу спагетти-ифовый чанкер, который, кажется, больше отвечает тз, чем предыдущий.

В него будут подаваться две строки (либо POS'ы, либо слово + POS) и третья строка, которая выбирает мод (pos, wordpos, posword). 
Теоретически, можно было бы сделать автоматическую подачу мода путём простого просмотра, есть ли поданное слово в списке POS (у нас выше есть converted_dict, можно было бы итерироваться по его items), но я не успеваю.

In [153]:
# available modes - pos, wordpos, posword
def chunker_dif(el1, el2, mode):
    bigram_list = []
    for word in words_annotated:
         # поиск по индексу следующего элемента
        if mode == "pos":
            if el1 == word[1]:
                next_word = words_annotated[words_annotated.index(word)+1] 
                if next_word[1] == el2:
                    bigram = word[0] + ' ' + next_word[0]
                    bigram_list.append(bigram)
        elif mode == "wordpos":
            if el1 == word[0]:
                next_word = words_annotated[words_annotated.index(word)+1] 
                if next_word[1] == el2:
                    bigram = word[0] + ' ' + next_word[0]
                    bigram_list.append(bigram)
        elif mode == "posword":
            if el1 == word[1]:
                next_word = words_annotated[words_annotated.index(word)+1] 
                if next_word[0] == el2: 
                    bigram = word[0] + ' ' + next_word[0]
                    bigram_list.append(bigram)
    return bigram_list

In [154]:
print(chunker_dif('S', 'S', 'pos'))

['вырей времирей', 'времушком камушком', 'времушек камушек', 'времушко камушко', 'времыня крылья', 'открыватель истин', 'круг прамин', 'прамин бога', 'кольцо потяти', 'бог длинноты', 'птицы вечности', 'кольцо потяти', 'боги чашу', 'волну истоков', 'истоков эксампей', 'главу дерзавицу', 'тело кружева', 'кружева изнанка', 'колыбели мотылька', 'жизни радуги', 'цвету север', 'север запад', 'южанки руку', 'березой ветвь', 'ветвь дева', 'указаний кремля', 'казани стены', 'стены битвою']


In [155]:
print(chunker_dif('целый', 'S', 'wordpos'))

['целый год']


In [156]:
print(chunker_dif('A', 'круг', 'posword'))

['томный круг']


Ну, выглядит это так, будто бы оно работает. 

В случае с отзывами, я бы предложила посмотреть на такие шаблоны:
- "не" + ADV/ADJ (не очень, не лучший/худший)
- "не" + VERB (не дотягивает, не работает) 
- ADJ + "но" (хороший, но/плохой, но) 
- ADV + ADJ (e.g. невероятно хороший)

Кажется, что отрицания в принципе должны встречаться чаще, но можно смотреть на предположительную тональность второго слова и в большинстве случаев инвертировать её (лучший -> не лучший -> отзыв скорее негативен). 

В случае с ADV + ADJ, кажется, в большинстве случаев оценка тональности коллокации будет совпадать с оцекой тональности ADV  (навскидку, такая комбинация будет встречаться в очень эмоциональных отзывах, но какой окраской они будут обладать, не вполне ясно + вещи вроде "ужасно интересный" - их тоже можно прописать)


(И нет, я не заимплементила это для предыдущей домашки, хотя очень хотелось :()